# Quickstart

In [ ]:
%matplotlib inline

In [ ]:
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Disable annoying font warnings
matplotlib.font_manager._log.setLevel(50)

# Disable theano deprecation warnings
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="theano")

# Style
plt.style.use("default")
plt.rcParams["savefig.dpi"] = 100
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (12, 4)
plt.rcParams["font.size"] = 14
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Liberation Sans"]
plt.rcParams["font.cursive"] = ["Liberation Sans"]
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["mathtext.fallback_to_cm"] = True

# Short arrays when printing
np.set_printoptions(threshold=0)

## Setup

Let us begin by importing the GP class we'll use, ``StarryProcess``:

In [ ]:
from starry_process import StarryProcess

as well as some standard stuff:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Because `starry_process` is written in just-in-time compiled `theano`, we'll need to explicitly compile the functions we want to use. Let's import some stuff to help us with that:

In [ ]:
import theano
import theano.tensor as tt

In order to visualize some surface maps, it's useful to also import the `starry` package:

In [ ]:
import starry

Let us instantiate a `StarryProcess` object with default values for everything:

In [ ]:
sp = StarryProcess()

The hyperparameters of the GP are as follows:

| attribute | description | default value |
| - | :- | :-:
| `r` | mean radius in degrees | `20` |
| `dr` | radius distribution half-width in degrees | `None`|
| `a` | latitude distribution shape parameter | `0.4` |
| `b` | latitude distribution shape parameter | `0.27` |
| `mu` | latitude distribution mode in degrees | `None` |
| `sigma` | latitude distribution standard deviation in degrees | `None` |
| `c` | fractional spot contrast | `0.1` |
| `n` | number of spots | `10` |

Two notes about these parameters.
First, by default, the radius distribution is a delta function centered at `r`; setting `dr`
to a numerical value changes this to a uniform distribution between `r - dr` and `r + dr`.
Second, by default, the latitude distribution is specified via the dimensionless shape
parameters `a` and `b`. These have a one-to-one correspondence with the mode `mu`
and standard deviation `sigma` of the distribution, which users can choose to specify instead.

Let's print the value of one of these hyperparameters as an example:

In [ ]:
sp.r

Not quite what we expected! That's because `starry_process` is built on `theano`, a just-in-time compiled graph-based language. Unless we explicitly evaluate variables, they are just nodes in a graph: instructions on *how* to perform a given operation. This may be a bit of a nuisance at times, but it's what makes `starry_process` so fast while enabling the automatic differentiation needed for integration with inference suites like `pymc3`.

Evaluating a `theano` tensor variable is super easy:

In [ ]:
sp.r.eval()

That's more like it! We can now check that all of the parameters are in fact set to their default values:

In [ ]:
for param in ["r", "dr", "a", "b", "mu", "sigma", "c", "n"]:
    print("{} = {}".format(param, getattr(sp, param).eval()))

Note that the `mu` and `sigma` parameters of the latitude distribution were automatically computed (since they are one-to-one functions of `a` and `b`). The default hyperparameters of the GP therefore correspond to 10 spots of radii $20^\circ$ at $30^\circ \pm 5^\circ$ latitude with 10% contrast.

## Sampling

### Sampling in spherical harmonics

Now that we've instantiated the GP, the simplest thing we can do is sample from it. There are two quantities we can sample: the surface map of the star and its corresponding light curve. The former is done by calling

In [ ]:
y = sp.sample_ylm().eval()

where `ylm` refers to the fact that the surface map is expressed in the spherical harmonic $Y_{l,m}$ basis, and we call `eval()` because, as before, we are dealing with `theano` tensors. The quantity `y` is the vector of spherical harmonic coefficients describing the surface:

In [ ]:
y

Its shape is `(number of samples, number of coefficients)`:

In [ ]:
y.shape

To get more samples, we could have used the `nsamples` kwarg to `sample_ylm()`. But since we only want one sample, let's reshape it into a 1d vector for convenience:

In [ ]:
y = y.reshape(-1)
y.shape

The vector has $(l_\mathrm{max} + 1)^2 = 256$ elements, since it is an expansion up to spherical harmonic degree $l_\mathrm{max} = 15$. The easiest way to visualize the correspnding surface map is by instantiating a `starry` `Map` object:

In [ ]:
map = starry.Map(15)
map[:, :] = y
map.show(projection="moll", colorbar=True)

We're seeing the surface in a Mollweide projection in relative units (i.e., the unspotted photosphere has an intensity of zero). The features cluster at about $\pm30^\circ$ latitude and have roughly the expected size and contrast. There are some bright features, however, and there aren't exactly 10 spots---this is all expected, since a stellar surface is not *really* a Gaussian process. What we're doing is just an approximation that works OK for sampling (but really well for inference).

To compute the observed light curve at some inclination, we could use `starry`:

In [ ]:
map.inc = 60
t = np.linspace(0, 4, 1000)
P = 1.0
flux = map.flux(theta=360 / P * t).eval()

where we set the inclination to $60^\circ$, the period to unity (in arbitrary units), and we're visualizing the light curve over four cycles. Let's normalize it and plot it:

In [ ]:
# Mean-normalize and convert to relative ppt
flux += 1
flux /= np.mean(flux)
flux -= 1
flux *= 1e3

# Plot
plt.plot(t, flux)
plt.xlabel("rotations")
plt.ylabel("relative flux [ppt]");

### Sampling in flux

If what we ultimately want are light curve samples, we can skip the map generation step entirely, and simply call

In [ ]:
flux = sp.sample(t, nsamples=50, eps=1e-9).eval()
flux

In [ ]:
flux.shape

where this time we asked for 50 samples evaluated at times `t` (defined above). The `eps` parameter is the extra variance we add to the covariance matrix for stability; sometimes it may be necessary to tweak this depending on the application.

Let's plot our samples:

In [ ]:
for k in range(50):
    plt.plot(t, 1e3 * flux[k], alpha=0.5)
plt.xlabel("rotations")
plt.ylabel("relative flux [ppt]");

By default, `StarryProcess` marginalizes over inclination. That means each of these samples corresponds to a different random inclination drawn from an isotropic distribution between $0^\circ$ and $90^\circ$ (the distribution is actually just $p(I) = \sin I$). If we wanted to condition on a specific inclination, we would pass `marginalize_over_inclination=False` when instantiating the `StarryProcess`, and explicitly pass a value for `i` in the call to `sample()`.

## More coming soon!